In [ ]:
// Initialization and nearest expiry
const debug = false;
const axios = require('axios');
const fs = require('fs');
const unzipper = require('unzipper');
const { parse } = require('papaparse');
const moment = require('moment');
let { authparams } = require("./cred");
const { idxNameTokenMap, downloadCsv, filterAndMapDates, 
  identify_option_type, fetchSpotPrice, getStrike, calcVix } = require('./utils/customLibrary');

let globalInput = {
  indexName: 'NIFTY',
  delayTime: 1000,
  ocGap: undefined,
  token: undefined,
  pickedExchange: undefined,
  inputOptTsym: undefined,
  WEEKLY_EXPIRY: undefined,
  MONTHLY_EXPIRY: undefined,
  filteredIndexCSV: undefined
};
globalInput.token = idxNameTokenMap.get(globalInput.indexName);

let biasProcess = {
  optionChain: undefined,
  atmCallSymbol: undefined,
  callSubStr: undefined,
  atmPutSymbol: undefined,
  putSubStr: undefined,
}
let biasOutput = { // N[46] 20155 (-20)
  tsym: '',
  bias: 0,
  deltaMove: 0,
  spotLP: 0
}

const delay = (ms) => new Promise(resolve => setTimeout(resolve, ms));

//websocket
let websocket;
let websocket_closed= false;
let intervalId;

let latestQuotes = {};
let latestOrders = {};


async function findNearestExpiry() {
  let csvUrl, zipFilePath, csvFilePath;
  const exchangeType = globalInput.indexName.includes('EX') ? 'BFO' : 'NFO';
  csvUrl = `https://api.shoonya.com/${exchangeType}_symbols.txt.zip`;
  zipFilePath = `./${exchangeType}_symbols.zip`;
  csvFilePath = `./${exchangeType}_symbols.txt`;
  try {
    // Download and extract the CSV file
    await downloadCsv(csvUrl, zipFilePath);
    await fs.createReadStream(zipFilePath).pipe(unzipper.Extract({ path: '.' }));

    // Read CSV data into a JavaScript object
    const csvData = fs.readFileSync(csvFilePath, 'utf-8');
    const { data: symbolDf } = parse(csvData, { header: true });
    
    globalInput.filteredIndexCSV = filterAndMapDates(symbolDf.filter((row) => ['OPTIDX', 'FUTIDX'].includes(row.Instrument) && row.TradingSymbol.startsWith(globalInput.indexName)));
    // console.log(globalInput.filteredIndexCSV);
    // [
    //  {
    //   Exchange: 'NFO',
    //   Token: '72903',
    //   LotSize: '50',
    //   Symbol: 'NIFTY',
    //   TradingSymbol: 'NIFTY29FEB24P21750',
    //   Expiry: '2024-02-29',
    //   Instrument: 'OPTIDX',
    //   OptionType: 'PE',
    //   StrikePrice: '21750',
    //   TickSize: '0.05',
    //   '': ''
    // },
    //BFO,833613,15,BKXFUT,BANKEX24FEBFUT,26-FEB-2024,FUTIDX,XX,0,0.05,
    const expiryList = [...new Set(globalInput.filteredIndexCSV.filter((row) => row.Instrument === 'OPTIDX').map((row) => row.Expiry))];
    const expiryFutList = globalInput.filteredIndexCSV
      .filter((row) => row.Instrument === 'FUTIDX')
      .map((row) => ({ Exchange: row.Exchange, LotSize: row.LotSize, TradingSymbol: row.TradingSymbol, Expiry: row.Expiry }));
    expiryList.sort();
    expiryFutList.sort((a, b) => moment(a.Expiry).diff(moment(b.Expiry)));
    
    globalInput.inputOptTsym = [...new Set(globalInput.filteredIndexCSV.filter((row) => (row.Instrument === 'OPTIDX' && row.Expiry === expiryList[0])).map((row) => row.TradingSymbol))][0];
    globalInput.WEEKLY_EXPIRY = expiryList[0];
    globalInput.MONTHLY_EXPIRY = expiryFutList[0].Expiry;
    globalInput.ocGap = expiryFutList[0].LotSize;
    globalInput.pickedExchange = expiryFutList[0].Exchange;
  } catch (error) {
    console.error('Error:', error.message);
  } finally {
    // Clean up: Delete downloaded files
    fs.unlinkSync(zipFilePath);
    fs.unlinkSync(csvFilePath);
  }
};
// Execute the findNearestExpiry function
findNearestExpiry();

const Api = require("./lib/RestApi");
let api = new Api({});

In [ ]:
// login method
login = async (api) => {
    try {
        const res = await api.login(authparams);
        return true;
    } catch (err) {
        return false;
    }
};
executeLogin = async () => {
    const isLoggedIn = await login(api);
    if (!isLoggedIn) {
        return;
    }
};
executeLogin();

In [ ]:
// websocket
function receiveQuote(data) {
    console.log("Quote ::", data);
    // Update the latest quote value for the corresponding instrument
    latestQuotes[data.e + '|' + data.tk] = data;
}

function receiveOrders(data) {
    console.log("Order ::", data);
    // Update the latest order value for the corresponding instrument
    latestOrders[data.Instrument] = data;
}

function open(data) {
    const initialInstruments = [`NSE|${globalInput.token}`];
    subscribeToInstruments(initialInstruments);
    console.log("Subscribing to :: ", initialInstruments);
}

function subscribeToInstruments(instruments) {
    instruments.forEach(instrument => {
        api.subscribe(instrument);
    });
}

function dynamicallyAddSubscription(newInstrument) {
    if (!latestQuotes[newInstrument]) {
        console.log("Subscribing to :: ", newInstrument);
        api.subscribe(newInstrument);
    }
}

const params = {
    'socket_open': open,
    'quote': receiveQuote,
    'order': receiveOrders
};
async function startWebsocket() {
    await delay(1000)
    websocket = api.start_websocket(params);
}
startWebsocket();


In [ ]:
// findATMfromOC
const getAtmStrike = async () => {
    // const s = await fetchSpotPrice(api, globalInput.token, globalInput.pickedExchange);
    await delay(1000);
    const s = latestQuotes[`NSE|${globalInput.token}`];
    debug && console.log(s) //updateAtmStrike(s) --> 50, spot object -> s.lp = 20100
    return Math.round(s.lp / globalInput.ocGap) * globalInput.ocGap
}

async function getOptionChain() {
    try {
        biasProcess.atmStrike = await getAtmStrike();
        const optionChainResponse = await api.get_option_chain(globalInput.pickedExchange, globalInput.inputOptTsym, biasProcess.atmStrike, 3);
        if (optionChainResponse.stat === 'Ok') {
            debug && console.log(optionChainResponse, 'optionChainResponse')
            return optionChainResponse;
        } else {
            console.error('Error getting option chain:', optionChainResponse);
            return null;
        }
    } catch (error) {
        console.error('Error:', error.message);
        return null;
    }
}

// Function to find the ATM symbol from the option chain
function findATMSymbol(optionType) {
    // Filter options by type (CE for Call, PE for Put)
    const filteredOptions = biasProcess.optionChain.values.filter(option => option.optt === optionType);

    // Assume that the ATM option is the one with the nearest strike price to the spot price
    const spotPrice = parseFloat(biasProcess.optionChain.values[0].strprc); // Use the first option's strike price as spotPrice
    filteredOptions.sort((a, b) => Math.abs(parseFloat(a.strprc) - spotPrice) - Math.abs(parseFloat(b.strprc) - spotPrice));

    // ATM symbol is the one with the minimum absolute difference
    const atmSymbol = filteredOptions[0].tsym;
    return atmSymbol;
}

async function findATMSymbolfromOC() {
    await delay(1000)
    // Get the Nifty option chain
    biasProcess.optionChain = await getOptionChain();

    if (biasProcess.optionChain) {
        // Find the ATM symbol
        biasProcess.atmCallSymbol = findATMSymbol('CE');
        biasProcess.atmPutSymbol = findATMSymbol('PE');
        debug && console.log(globalInput)
        debug && console.log(biasProcess)
        debug && console.log(biasOutput)
    }
}

// Run the example
findATMSymbolfromOC();

In [ ]:
// Print the latest values every 5 seconds
function myRecurringFunction() {
    console.log(`${globalInput.indexName}:`, latestQuotes[`NSE|${globalInput.token}`] ? latestQuotes[`NSE|${globalInput.token}`].lp : "N/A", "Order:", latestOrders[`NSE|${globalInput.token}`]);
    console.log(`${biasProcess.atmCallSymbol}:`, latestQuotes[biasProcess.callSubStr] ? latestQuotes[biasProcess.callSubStr].lp : "N/A", "Order:", latestOrders[biasProcess.callSubStr]);
    console.log(`${biasProcess.atmPutSymbol}:`, latestQuotes[biasProcess.putSubStr] ? latestQuotes[biasProcess.putSubStr].lp : "N/A", "Order:", latestOrders[biasProcess.putSubStr]);

    // Check a condition to determine whether to stop the recurring function
    if (websocket_closed) {
      clearInterval(intervalId);
      console.log('Recurring function stopped.');
    }
}

// Start the recurring function and store the interval identifier
intervalId = setInterval(myRecurringFunction, 5000);


function getTokenByTradingSymbol(tradingSymbol) {
    const option = biasProcess.optionChain.values.find(option => option.tsym === tradingSymbol);
    if (option) {
      return option.token;
    } else {
      return null; // TradingSymbol not found
    }
  }
  // console.log(getTokenByTradingSymbol(biasProcess.atmCallSymbol));
// console.log(`NFO|${getTokenByTradingSymbol(biasProcess.atmCallSymbol)}`);


// Example: Dynamically add a subscription after 10 seconds
setTimeout(() => {
  biasProcess.callSubStr = `NFO|${getTokenByTradingSymbol(biasProcess.atmCallSymbol)}`;
  biasProcess.putSubStr = `NFO|${getTokenByTradingSymbol(biasProcess.atmPutSymbol)}`;
    dynamicallyAddSubscription(biasProcess.callSubStr);
    dynamicallyAddSubscription(biasProcess.putSubStr);
}, 10000);

// Example: Dynamically add a subscription after 10 seconds
setTimeout(() => {
  websocket.close();
  websocket_closed = true;
}, 20000);
